In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

Mounted at /content/drive
/content


In [3]:
!ls
%cd drive/MyDrive/clothing_project
!pwd

drive  sample_data
/content/drive/MyDrive/clothing_project
/content/drive/MyDrive/clothing_project


In [3]:
!pip install bs4
!pip install lxml
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=2b97457baa32738c6eacd47ca53d86d07be27019dab3af5b9193c99dff354d0e
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Web-scraping from Amazon

In [4]:
from bs4 import BeautifulSoup as blsp
import requests
import time
import csv

In [39]:
links_file = open('links.txt', 'r')
links = links_file.readlines()

In [ ]:
csv_rows = []
dataset_file = open("cloth_dataset.csv", 'w')
csvwriter = csv.writer(dataset_file) 
fields = ['brand', 'description', 'link'] 
csvwriter.writerow(fields)
count = 1
for link in links:
  webpage = requests.get(link)
  time.sleep(10)
  page_html = blsp(webpage.content, "lxml")
  cloth_blocks = page_html.findAll("div",{"class":"a-section a-spacing-none a-spacing-top-small s-title-instructions-style"})
  for block in cloth_blocks:
    try:
      cbrand = block.find("span",{"class":"a-size-base-plus a-color-base"}).string.strip()
    except:
      cbrand = None
    try:
      cdescp = block.find("span",{"class":"a-size-base-plus a-color-base a-text-normal"}).string.strip()
    except:
      cdescp = None
    try:
      clink = "https://www.amazon.in"+block.find("a",{"class":"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"}).get("href").strip()
    except:
      clink = None
    csvwriter.writerow([cbrand, cdescp, clink])
    csv_rows.append([cbrand, cdescp, clink])
  count += 1
dataset_file.close()

In [5]:
!pip install PyStemmer
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyStemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-linux_x86_64.whl size=584860 sha256=d5fa8603fe4b4507846c663471d47571cacf65291892249287dca62ab80efe66
  Stored in directory: /root/.cache/pip/wheels/45/7d/2c/a7ebb8319e01acc5306fa1f8558bf24063d6cec2c02de330c9
Successfully built PyStemmer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Text Processing

In [6]:
import string
import nltk
from nltk.stem import WordNetLemmatizer
from Stemmer import Stemmer
from nltk.corpus import stopwords as nltk_stopwords
import time
import sys
import re
import csv
import pandas as pd
import tensorflow_hub as hub

nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))
url_stopwords = ["www", "com", "http", "https", "net", "org", "html", "ftp", "archives", "pdf", "jpg", "jpeg", "gif", "png", "txt", "redirect"]
punctuations = list(string.punctuation) + ['_', '\t', '\n']
digits = [str(x) for x in range(10)]
max_wordlen = 20
min_wordlen = 3
use_lematizer = True
output_dir = "."
stemmer = Stemmer('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def is_english(word):
  try:
    str(word).encode(encoding='utf-8').decode('ascii')
  except UnicodeDecodeError:
    return False
  else:
    return True

def tokenize(data):
  global min_wordlen, max_wordlen
  good_tokens = []
  rpun = '[' + re.escape(''.join(punctuations)) + ']'
  data = re.sub(rpun, ' ', data.strip())
  tokens = data.strip().split()
  for t in tokens:
    if len(str(t))>=min_wordlen and len(str(t))<max_wordlen and is_english(t):
      good_tokens.append(t)
  return good_tokens

def remove_stopwords(words):
  good_words = []
  for w in words:
    if w in stopwords or w in url_stopwords:
      continue
    good_words.append(w)
  return good_words

def process_words(words):
  global min_wordlen, max_wordlen
  words = remove_stopwords(words)
  words = [stemmer.stemWord(w) for w in words]
  words = [w for w in words if len(w)>=min_wordlen and len(w)<max_wordlen]
  return words

def process_text(content):
  content = content.lower()
  tokens = tokenize(content)
  words = process_words(tokens)
  return words

In [8]:
dataset = pd.read_csv("cloth_dataset.csv")
dataset.head()

,brand,description,link
0,VSTAR,Women's Viscose Plain Half Sleeve Comfy Blouse,https://www.amazon.in/VSTAR-Blouse-Perfect-Rea...
1,Jockey,1301 Women's Super Combed Cotton Elastane Stre...,https://www.amazon.in/Jockey-Womens-Relaxed-13...
2,Amayra,Women's A-Line Knee Length Dress,https://www.amazon.in/Amayra-Womens-Rayon-Line...
3,DUMMY SHAPE,Women's Cotton Rayon Green Maternity & Feeding...,https://www.amazon.in/DUMMY-SHAPE-Womens-Mater...
4,Womanista,Women's Floral Printed Poly Georgette Saree wi...,https://www.amazon.in/Womanista-georgette-blou...


In [ ]:
processed_descp = []
for i, row in dataset.iterrows():
  descp = row['brand']+" "+row['description']
  descp = process_text(descp)
  descp = ' '.join(descp)
  processed_descp.append(descp)

In [13]:
df = pd.DataFrame({'processed_text':processed_descp})
dataset['processed_text'] = df['processed_text']
dataset.to_csv("pro_cloth_dataset.csv")

#Text Embedding

In [9]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

In [ ]:
all_embeds = []
for i, row in dataset.iterrows():
  embed = model([row['processed_text']])
  all_embeds.append(embed)

In [20]:
import numpy as np

all_embeds = np.array(all_embeds)
all_embeds.shape

(96, 1, 512)

In [21]:
np.save('all_embeds.npy', all_embeds)

#Inference Using Product Similarity Between Embeddings

In [10]:
import numpy as np
all_embeds = np.load('all_embeds.npy')

In [ ]:
input = "zeyo night suit"
descp = process_text(input)
descp = ' '.join(descp)

embed = model([descp])

top5 = np.inner(embed, all_embeds)
top5_index = np.argsort(-top5,-2)[:,:5,:]

In [ ]:
top5_index = np.squeeze(np.squeeze(top5_index,-1),0)

In [53]:
dataset.iloc[top5_index]

,brand,description,link
24,ZEYO,Women's Cotton Star Printed Night Suit Set of ...,https://www.amazon.in/ZEYO-Womens-Cotton-Print...
18,ZEYO,Women's Cotton Flash Printed Night Suit Set of...,https://www.amazon.in/ZEYO-Womens-Printed-Styl...
26,ZEYO,Women's Cotton Strip Printed Night Suit Set of...,https://www.amazon.in/ZEYO-Lavender-Striped-Pr...
30,ZEYO,Women's Cotton 3PC's Heart Printed Night Suit ...,https://www.amazon.in/ZEYO-Womens-Cotton-Trian...
35,ZEYO,Women's Cotton Dot Printed Night Suit Set of T...,https://www.amazon.in/ZEYO-Womens-Cotton-Print...
